In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

In [ ]:
import os

In [ ]:
df = pd.read_parquet(os.getcwd() + "/files/all_df_combined.parquet")

In [ ]:
carrier_list = df['md_carrier'].tolist()
carrier_list = [x for x in carrier_list if x is not None]

pp_carrier_list = df['pp_carrier'].tolist()
pp_carrier_list = [x for x in pp_carrier_list if x is not None]

In [ ]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(["llc", "dba", "corp"])

vectorizer = TfidfVectorizer(stop_words=my_stop_words)
X = vectorizer.fit_transform(carrier_list)

In [ ]:
true_k = 30
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100,
               n_init=10)
model.fit(X)

In [ ]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [ ]:
order_centroids

In [ ]:
for i in range(true_k): # look at cluster keywords
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])

In [ ]:
#new_df = df[['acctno','gathered_ph','md_carrier']][df['md_carrier'].is.null()]
md_df = df[df['md_carrier'].isin(carrier_list)]
md_df = md_df[['acctno','gathered_ph','md_carrier']]

pp_df = df[df['pp_carrier'].isin(carrier_list)]
pp_df = pp_df[['acctno','gathered_ph','pp_carrier']]

md_df.shape, pp_df.shape

In [65]:
X = vectorizer.transform(md_df['md_carrier'].tolist())
predicted = model.predict(X)

In [67]:
X_pp = vectorizer.transform(pp_df['pp_carrier'].tolist())
pp_predicted = model.predict(X_pp)

In [68]:
md_df['md_carrier_group_mod'] = predicted
pp_df['mpp_carrier_group_mod'] = pp_predicted

In [69]:
new_df = pd.merge(md_df, pp_df, how = "outer", on = ["acctno", "gathered_ph"])

In [70]:
new_df.head(10)

,acctno,gathered_ph,md_carrier,md_carrier_group_mod,pp_carrier,mpp_carrier_group_mod
0,2101010001,2092279114,SPRINT SPECTRUM L.P.,6.0,NaN,NaN
1,2101010001,2092304725,"NEW CINGULAR WIRELESS PCS, LLC - GA",4.0,NaN,NaN
2,2101010001,2094066813,ATT,2.0,NaN,NaN
3,2101010001,2094634034,PACIFIC BELL,1.0,PACIFIC BELL,1.0
4,2101010001,2094655806,PACIFIC BELL,1.0,PACIFIC BELL,1.0
5,2101010001,2094667929,PACIFIC BELL,1.0,PACIFIC BELL,1.0
6,2101010001,2094673924,"UTILITY TELECOM GROUP, LLC - CA",2.0,NaN,NaN
7,2101010001,2094690878,PACIFIC BELL,1.0,PACIFIC BELL,1.0
8,2101010001,2094698944,PACIFIC BELL,1.0,PACIFIC BELL,1.0
9,2101010001,2094702745,CELLCO PARTNERSHIP DBA VERIZON WIRELESS - CA,10.0,NaN,NaN


In [71]:
new_df.to_csv(os.getcwd()+"./files/carrier_mod_groups.csv")

In [ ]:
TfidfVectorizer?